In [8]:
import osmnx as ox
import networkx as nx
import pandas as pd
from geopy.geocoders import Nominatim, GoogleV3
import time
import folium

In [1]:
coords

NameError: name 'coords' is not defined

In [ ]:
m = folium.Map(location=[55.7828289, 37.6988399],
              zoom_start=15)

loc = coords

folium.PolyLine(loc,
                color='red',
                weight=1,
                opacity=0.8).add_to(m)
m

In [ ]:
ox.config(log_console=True, use_cache=True)
print(1)
G_walk = ox.graph_from_place('Moscow, Russia',
                             network_type='walk')
print(1)
orig_node = ox.get_nearest_node(G_walk,
                                (55.7613387, 37.6573543))
print(1)
dest_node = ox.get_nearest_node(G_walk,
                                (55.7321928, 37.673857749999996))
print(1)
route = nx.shortest_path(G_walk,
                         orig_node,
                         dest_node,
                         weight='length')
print(1)
route_map = ox.plot_route_folium(G_walk, route)
print(1)
route_map.save('route.html')

In [34]:
route_map

NameError: name 'route_map' is not defined

In [ ]:
ox.config(log_console=True, use_cache=True)
# define the start and end locations in latlng
# start_latlng = (37.78497,-122.43327)
# end_latlng = (37.78071,-122.41445)
# location where you want to find your route
place     = 'San Francisco, California, United States'
# find shortest route based on the mode of travel
mode      = 'walk'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'
# create graph from OSM within the boundaries of some 
# geocodable place(s)
graph = ox.graph_from_place(place, network_type = mode)
# find the nearest node to the start location
orig_node = ox.get_nearest_node(graph, start_latlng)
# find the nearest node to the end location
dest_node = ox.get_nearest_node(graph, end_latlng)
#  find the shortest path
shortest_route = nx.shortest_path(graph,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer)

In [ ]:
# shortest_route_map = ox.plot_route_folium(graph, shortest_route)
graph

In [6]:
address_df = pd.read_excel('source.xlsx')

In [84]:
def unifier(x: str):
    x = x.replace('1-ая','1').replace('1-я','1').replace('пер.','переулок')
    x = x.replace('корп.','к.').replace(' строение','/').replace('-','/').replace(' стр.','/')
    x = x.replace(' стр','/').replace('бульв.','бульвар').replace('Нов.','Новая').replace('Стар.','Старая')
    list_to_del = ['наб.','Б.','М.','ул.','д.',',','БП', ' - БП']
    for i in list_to_del:
         x = x.replace(i,'')
    return x

In [11]:
def get_coords(address: str):
    try:
        geolocator = GoogleV3()
        location = 'Россия, Москва, '+address
        print(location)
        location = geolocator.geocode(location)
        
        
        return location.latitude, location.longitude
    except:
        return 'bad address'

In [ ]:
def get_coords2(address: str):
    try:
        geolocator = Nominatim(user_agent="parser")
        adr_str = unifier(address)
        print(adr_str)
        location = geolocator.geocode(adr_str)
        check = str(location).replace(' ','').split(',')
        if 'Москва' in check:
            return location.latitude, location.longitude
        else:
            try:
                adr_str = 'Москва '+adr_str
                location = geolocator.geocode(adr_str)
                return location.latitude, location.longitude
            except:
                for i in range(len(adr_str)):
                    adr_str = adr_str[0:-1]
                    try:
                        print(adr_str)
                        location = geolocator.geocode('Москва '+adr_str)
                        return location.latitude, location.longitude
                        break
                    except:
                        print('bad address') 
    except:
        print('bad address')
        return 'bad address'

In [86]:
def get_coords_temp_main(address: str):
    geolocator = Nominatim(user_agent="parser")
    adr_str = unifier(address)
    print(adr_str)
    location = geolocator.geocode(adr_str)
    # if location.longitude > 39.0:
    #     adr_str = 'Москва, ул. '+adr_str
    #     location = geolocator.geocode(adr_str)
    # elif location.latitude > 39.0:
    #     adr_str = 'Москва, ул. '+adr_str
    #     location = geolocator.geocode(adr_str)
    check = str(location).replace(' ','').split(',')
    if 'Москва' in check and location.longitude < 39.0 and location.latitude < 59.0:
        return [location.latitude, location.longitude]
    else:
        
        try:
            if 'переулок' not in adr_str.split(' ') and 'пр.' not in adr_str.split(' '):
                adr_str = 'Москва, ул. '+ adr_str
                print(adr_str)
            else:
                 
                adr_str = 'Москва, '+adr_str
            location = geolocator.geocode(adr_str)
            return [location.latitude, location.longitude]
        except:
            for i in range(len(adr_str)):
                adr_str = adr_str[0:-1]
                try:
                    print(adr_str)
                    location = geolocator.geocode(adr_str)
                    return [location.latitude, location.longitude]
                    break
                except:
                    pass
            # return [location.latitude, location.longitude]
                

In [12]:
address_df['coords'] = address_df['Адрес объекта исходный'].apply(lambda x: get_coords(x))
address_df

,Адрес объекта исходный,coords
0,Рубцовская наб. 2 корп.1,bad address
1,Иерусалимский пр. 4,bad address
2,"Калитниковская М. ул., д. 45",bad address
3,"Иерусалимская ул., д. 6",bad address
4,Почтовая Б. ул. 18,bad address
...,...,...
92,Доброслободская ул. 4,bad address
93,"Котельническая набережная, 33 к.1 - БП",bad address
94,"Рабочая ул., д. 4",bad address
95,"Земляной вал, д. 42/20",bad address


In [ ]:
wrong = address_df[address_df['coords']=='bad address']
wrong

In [ ]:
wrong

In [ ]:
wrong['coords'] = wrong['Адрес объекта исходный'].apply(lambda x: get_coords3(x))

In [29]:
pd.set_option("display.max_rows", None)
address_df

,Адрес объекта исходный,coords
0,Рубцовская наб. 2 корп.1,"[55.7828289, 37.6988399]"
1,Иерусалимский пр. 4,"[55.73253965, 37.67521092238345]"
2,"Калитниковская М. ул., д. 45","[55.7347111, 37.6840339]"
3,"Иерусалимская ул., д. 6","[55.73240455, 37.67714199013598]"
4,Почтовая Б. ул. 18,"[55.24333105, 42.21012679582428]"
5,"Библиотечная ул., д. 15/8","[55.7447711, 37.6717036787725]"
6,"Таганская ул., д. 31/22","[55.74012205, 37.67091024198514]"
7,Доброслободская ул. 11/13,"[55.76739275, 37.673993796349976]"
8,"Каменщики Б. ул., д. 8","[55.7352928, 37.656038]"
9,"Подколокольный пер., д.16/2 стр.1","[55.7527803, 37.6445521]"


In [85]:
geolocator = Nominatim(user_agent="parser")
location = 'Москва, ул. Жуковского  2'
location = geolocator.geocode(location)
print([location.latitude, location.longitude])
print(location)

[55.76386635, 37.64478838887051]
2, улица Жуковского, Басманный район, Москва, Центральный федеральный округ, 101000, Россия


In [34]:
a = get_coords_temp_main('Нижегородская   9а')
a

Нижегородская   9а
Москва, ул. Нижегородская   9а


[56.11795935, 43.49617445]

In [29]:
location.raw

{'place_id': 113001904,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 45130528,
 'boundingbox': ['55.7645879', '55.7647939', '37.6535577', '37.6540234'],
 'lat': '55.7646852',
 'lon': '37.653772366975886',
 'display_name': '21, улица Машкова, Басманный район, Москва, Центральный федеральный округ, 105062, Россия',
 'class': 'building',
 'type': 'apartments',
 'importance': 0.42099999999999993}

In [ ]:
adr_str = 'Москва Земляной Вал 27'
for i in range(len(adr_str)):
    adr_str = adr_str[0:-1]
    try:
        print(adr_str)
        location = geolocator.geocode('Москва, ул. '+adr_str)
        print(location.latitude, location.longitude)
        break
    except:
        print('bad address')
         

In [ ]:
try:
    geolocator = Nominatim(user_agent="parser")
    string = 'Иерусалимский  4'
    print(string)
    location = geolocator.geocode(string)
    check = str(location).replace(' ','').split(',')
    if 'Москва' in check:
        print(location.latitude, location.longitude)
    else:
        try:
            location = geolocator.geocode('Москва '+string)
            print(location.latitude, location.longitude)
        except:
            print('bad address') 
except:

    print('bad address') 

In [ ]:
location = geolocator.geocode('Москва '+str(location))

In [81]:
coords = address_df['coords'].to_list()
for i in coords:
    if 55 > i[0] > 56 or 37 < i[1] > 38:
        print(f'Deleted {i}: wrong address')
        coords.remove(i)
coords

Deleted [55.24333105, 42.21012679582428]: wrong address
Deleted [55.8024135, 38.9910508]: wrong address
Deleted [55.4627124, 42.318168749976394]: wrong address
Deleted [55.4627124, 42.318168749976394]: wrong address
Deleted [56.11795935, 43.49617445]: wrong address
Deleted [55.47178845, 43.20845997012833]: wrong address
Deleted [55.32913295, 42.013658916235556]: wrong address


[[55.7828289, 37.6988399],
 [55.73253965, 37.67521092238345],
 [55.7347111, 37.6840339],
 [55.73240455, 37.67714199013598],
 [55.7447711, 37.6717036787725],
 [55.74012205, 37.67091024198514],
 [55.76739275, 37.673993796349976],
 [55.7352928, 37.656038],
 [55.7527803, 37.6445521],
 [55.76016905, 37.64752874688179],
 [55.7328116, 37.6776434],
 [55.763170599999995, 37.65619941154437],
 [55.7351149, 37.6675703],
 [55.76386635, 37.64478838887051],
 [55.7342582, 37.6542835],
 [55.7613387, 37.6573543],
 [55.7547934, 37.6510301],
 [55.73713, 37.6482834],
 [55.739151, 37.6694611],
 [55.73884275, 37.68057808904321],
 [55.7586866, 37.657150099999996],
 [55.731397599999994, 37.683352717243764],
 [55.7322019, 37.680129294263935],
 [55.782835, 37.7302759],
 [55.743725100000006, 37.678069453189494],
 [55.7578288, 37.65612170576381],
 [55.7614943, 37.6627771],
 [55.742128199999996, 37.683084798388535],
 [55.75525745, 37.655339935775025],
 [55.7611996, 37.645871],
 [55.766234749999995, 37.6680134446625

In [ ]:
from collections import defaultdict
from math import sqrt

# Shortest path to all coordinates from any node
# Coordinates must be provided as a list containing lists of
# x/y pairs. ie [[23.2321, 58.3123], [x.xxx, y.yyy]]


def distance_between_coords(x1, y1, x2, y2):
    distance = sqrt(((x2 - x1) ** 2) + ((y2 - y1) ** 2))
    return distance


# Adds "names" to coordinates to use as keys for edge detection
def name_coords(coords):
    coord_count = 0
    for coord in coords:
        coord_count += 1
        coord.append(coord_count)
    return coords


# Creates a weighted and undirected graph
# Returns named coordinates and their connected edges as a dictonary
def graph(coords):
    coords = name_coords(coords)
    graph = defaultdict(list)
    edges = {}
    for current in coords:
        for comparer in coords:
            if comparer == current:
                continue
            else:
                weight = distance_between_coords(current[0], current[1],
                                                 comparer[0], comparer[1])
                graph[current[2]].append(comparer[2])
                edges[current[2], comparer[2]] = weight
    return coords, edges


# Returns a path to all nodes with least weight as a list of names
# from a specific node
def shortest_path(node_list, edges, start):
    neighbor = 0
    unvisited = []
    visited = []
    total_weight = 0
    current_node = start
    for node in node_list:
        if node[2] == start:
            visited.append(start)
        else:
            unvisited.append(node[2])
    while unvisited:
        for index, neighbor in enumerate(unvisited):
            if index == 0:
                current_weight = edges[start, neighbor]
                current_node = neighbor
            elif edges[start, neighbor] < current_weight:
                current_weight = edges[start, neighbor]
                current_node = neighbor
        total_weight += current_weight
        unvisited.remove(current_node)
        visited.append(current_node)
    return visited, total_weight


def driver(coords):
    coords, edges = graph(coords)
    shortest_path(coords, edges, 3)
    shortest_path_taken = []
    shortest_path_weight = 0

    for index, node in enumerate(coords):
        path, weight = shortest_path(coords, edges, index + 1)
        print('--------------------------------------')
        print("Path", index + 1, "=", path)
        print("Weight =", weight)
        if index == 0:
            shortest_path_weight = weight
            shortest_path_taken = path
        elif weight < shortest_path_weight:
            shortest_path_weight = weight
            shortest_path_taken = path
    print('--------------------------------------')
    print("The shortest path to all nodes is:", shortest_path_taken)
    print("The weight of the path is:", shortest_path_weight)

In [ ]:
driver(coords)

In [ ]:
def address_parser(x: str):
    x_parsed = x.replace(' корп.','/').replace('М.','Малая').replace('Б.','Большая').replace(' к.','/').replace(' - БП','').replace('пер.','переулок').replace(' стр.','/').replace('д.','').replace(',','').replace('-','/')
    city = 'Москва'
    x_full = city+' '+x_parsed
    print(x_full)
    geolocator = Nominatim(user_agent="parser")
    location = x_parsed
    location = geolocator.geocode(location)
    return location.latitude, location.longitude

In [ ]:
print(address_parser('каменщики 6'))

In [ ]:
address_df

In [ ]:
%%time
address_df['coords'] = address_df['Адрес объекта исходный'].apply(lambda x: address_parser(x))

In [ ]:
address_df

In [ ]:
geolocator = Nominatim(user_agent="xls_parser")
# location = ("Москва,  малая Калитниковская ул., д. 45")
location = geolocator.geocode('Москва Калитниковская Малая ул., д. 45')
print(location.address)
# print((location.latitude, location.longitude))
# # (40.7410861, -73.9896297241625)

In [ ]:
location = ("Калитниковская ул., д. 45")
a = location.split()

In [ ]:
a

In [ ]:
add = 'Калитниковская М. ул., д. 45'
add = add.replace('М.','Малая',1)
add2 = 'Рубцовская наб. 2 корп.1'
add2 = add2.replace(' корп.','/')
add2

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="xls_parser")
location = geolocator.geocode("Рубцовская наб 2/1")
print(location.address)
# Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
# >>> print((location.latitude, location.longitude))
# (40.7410861, -73.9896297241625)

In [ ]:
import requests

GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

params = {
    'address': 'oshiwara industerial center goregaon west mumbai',
    'sensor': 'false',
    'region': 'india'
}

# Do the request and get the response data
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()

# Use the first result
result = res['results'][0]

geodata = dict()
geodata['lat'] = result['geometry']['location']['lat']
geodata['lng'] = result['geometry']['location']['lng']
geodata['address'] = result['formatted_address']

print('{address}. (lat, lng) = ({lat}, {lng})'.format(**geodata))

# Result => Link Rd, Best 

In [ ]:
from pygeocoder import Geocoder

business_name = "Workafella Busswdmapminess Centre - Hitec city"
print ("Searching %s" %business_name)
results = Geocoder.geocode(business_name)
for result in results:
    print (result)